In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
xgb_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_mrs02/with_feature_aggregration/testing/test_gt_and_pred.pkl'
transformer_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_mrs02/transformer_20230402_184459_test_set_evaluation/fold_2_test_gt_and_pred.pkl'
mlp_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/mrs02/with_feature_aggregration_mlp/all_folds/test_gt_and_pred_cv_2.pkl'
lstm_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/testing/3M_mRS02/2023_01_02_1057/test_LSTM_sigmoid_all_balanced_0.2_2_True_RMSprop_3M mRS 0-2_16_3/test_gt_and_pred.pkl'
thrivec_mrs_predictions_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_3m_mrs02_predictions/test_gt_and_pred.pkl'

xgb_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_Death/testing/test_gt_and_pred.pkl'
transformer_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_Death/testing/fold_1_test_gt_and_pred.pkl'
mlp_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/death/feature_aggregration_mlp/testing/test_gt_and_pred_cv_2.pkl'
lstm_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/testing/3M_Death/2023_01_04_2020/all_folds/test_LSTM_sigmoid_all_balanced_0.0_2_True_RMSprop_3M Death_128_2/test_gt_and_pred_fold_2.pkl'
thrivec_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_3m_death_predictions/3m_death_test_gt_and_pred.pkl'

xgb_death_in_hospital_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/Death_in_hospital/testing/test_gt_and_pred_cv_2.pkl'
transformer_death_in_hospital_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/Death_in_hospital/testing/15_02_23/fold_2_test_gt_and_pred.pkl'
lstm_death_in_hospital_predictions_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/testing/Death_in_hospital/2024_02_05_1346/LSTM_test_death_in_hospital/test_LSTM_sigmoid_all_unchanged_0.2_1_True_RMSprop_Death in hospital_32_3/test_gt_and_pred_fold_3.pkl'
thrive_c_death_in_hosp_predictions_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_death_in_hospital_predictions/death_in_hospital_test_gt_and_pred.pkl'

xgb_mimic_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_Death/external_validation/external_validation_gt_and_pred.pkl'
transformer_mimic_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_Death/external_validation/fold_1_test_gt_and_pred.pkl'
mlp_mimic_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/death/feature_aggregration_mlp/external_validation/fold_2_test_gt_and_pred.pkl'
lstm_mimic_death_predictions_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/external_validation/3M_Death/2023_01_04_2020/all_folds/test_LSTM_sigmoid_all_balanced_0.0_2_True_RMSprop_3M Death_128_2/test_gt_and_pred_fold_2.pkl'

xgb_mimic_death_in_hospital_predictions_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/Death_in_hospital/external_validation/external_validation_gt_and_pred_cv_2.pkl' 
transformer_mimic_death_in_hospital_predictions_path = '/Users/jk1/temp/opsum_prediction_output/transformer/Death_in_hospital/external_validation/fold_2_test_gt_and_pred.pkl'
lstm_mimic_death_in_hospital_predictions_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/external_validation/Death_in_hospital/2024_02_24/test_LSTM_sigmoid_all_unchanged_0.2_1_True_RMSprop_Death in hospital_32_3/test_gt_and_pred_fold_3.pkl'

# Compare transformer and XGB

In [ ]:
from sklearn.metrics import roc_curve, auc
# Create a table with the AUC of the two models for mrs / death and mimic (with p-values)

import pandas as pd
import numpy as np
from prediction.outcome_prediction.model_comparison.de_long_comparison import delong_roc_test

def get_auc(gt, predictions):
    fpr, tpr, _ = roc_curve(gt, predictions)
    return auc(fpr, tpr)

def get_auc_and_p_value(gt, other_model_predictions, transformer_predictions):
    auc_other_model = get_auc(gt, other_model_predictions)
    auc_transformer = get_auc(gt, transformer_predictions)
    p_value = 10 ** delong_roc_test(gt, other_model_predictions, transformer_predictions)
    return auc_other_model, auc_transformer, p_value

def get_auc_and_p_value_for_outcome(outcome):
    if outcome == '3M Death':
        xgb_predictions_path = xgb_death_predictions_path
        transformer_predictions_path = transformer_death_predictions_path
    elif outcome == '3M mRS 0-2':
        xgb_predictions_path = xgb_mrs_predictions_path
        transformer_predictions_path = transformer_mrs_predictions_path
    elif outcome == 'Death in hospital':
        xgb_predictions_path = xgb_death_in_hospital_predictions_path
        transformer_predictions_path = transformer_death_in_hospital_predictions_path
    elif outcome == 'MIMIC 3M Death':
        xgb_predictions_path = xgb_mimic_death_predictions_path
        transformer_predictions_path = transformer_mimic_death_predictions_path
    elif outcome == 'MIMIC Death in hospital':
        xgb_predictions_path = xgb_mimic_death_in_hospital_predictions_path
        transformer_predictions_path = transformer_mimic_death_in_hospital_predictions_path
    else:
        raise ValueError('Unknown outcome')

    xgb_gt, xgb_predictions = pickle.load(open(xgb_predictions_path, 'rb'))
    transformer_gt, transformer_predictions = pickle.load(open(transformer_predictions_path, 'rb'))

    auc_xgb, auc_transformer, p_value = get_auc_and_p_value(xgb_gt, xgb_predictions, transformer_predictions)
    return auc_xgb, auc_transformer, p_value

df = pd.DataFrame(columns=['Outcome', 'XGB AUC', 'Transformer AUC', 'p-value'])
for outcome in ['Death in hospital', '3M Death', '3M mRS 0-2', 'MIMIC Death in hospital','MIMIC 3M Death']:
    auc_xgb, auc_transformer, p_value = get_auc_and_p_value_for_outcome(outcome)
    df = df.append({'Outcome': outcome, 'XGB AUC': auc_xgb, 'Transformer AUC': auc_transformer, 'p-value': p_value[0][0]}, ignore_index=True)


In [ ]:
df

In [ ]:
# df.to_csv('/Users/jk1/Downloads/model_comparison_xgb_transformer.csv')

# Compare MLP with transformer

In [ ]:
def get_auc_and_p_value_for_outcome_mlp(outcome):
    if outcome == '3M Death':
        mlp_predictions_path = mlp_death_predictions_path
        transformer_predictions_path = transformer_death_predictions_path
    elif outcome == '3M mRS 0-2':
        mlp_predictions_path = mlp_mrs_predictions_path
        transformer_predictions_path = transformer_mrs_predictions_path

    elif outcome == 'MIMIC 3M Death':
        mlp_predictions_path = mlp_mimic_death_predictions_path
        transformer_predictions_path = transformer_mimic_death_predictions_path
    else:
        raise ValueError('Unknown outcome')

    mlp_gt, mlp_predictions = pickle.load(open(mlp_predictions_path, 'rb'))
    transformer_gt, transformer_predictions = pickle.load(open(transformer_predictions_path, 'rb'))

    auc_mlp, auc_transformer, p_value = get_auc_and_p_value(mlp_gt, mlp_predictions, transformer_predictions)
    return auc_mlp, auc_transformer, p_value

df = pd.DataFrame(columns=['Outcome', 'MLP AUC', 'Transformer AUC', 'p-value'])
for outcome in ['3M Death', '3M mRS 0-2', 'MIMIC 3M Death']:
    auc_mlp, auc_transformer, p_value = get_auc_and_p_value_for_outcome_mlp(outcome)
    df = df.append({'Outcome': outcome, 'MLP AUC': auc_mlp, 'Transformer AUC': auc_transformer, 'p-value': p_value[0][0]}, ignore_index=True)

In [ ]:
df

In [ ]:
# df.to_csv('/Users/jk1/Downloads/model_comparison_mlp_transformer.csv')

# Compare LSTM with transformer

In [ ]:
def get_auc_and_p_value_for_outcome_lstm(outcome):
    if outcome == '3M Death':
        lstm_predictions_path = lstm_death_predictions_path
        transformer_predictions_path = transformer_death_predictions_path
    elif outcome == '3M mRS 0-2':
        lstm_predictions_path = lstm_mrs_predictions_path
        transformer_predictions_path = transformer_mrs_predictions_path
    elif outcome == 'Death in hospital':
        lstm_predictions_path = lstm_death_in_hospital_predictions_path
        transformer_predictions_path = transformer_death_in_hospital_predictions_path
    elif outcome == 'MIMIC 3M Death':
        lstm_predictions_path = lstm_mimic_death_predictions_path
        transformer_predictions_path = transformer_mimic_death_predictions_path
    elif outcome == 'MIMIC Death in hospital':
        lstm_predictions_path = lstm_mimic_death_in_hospital_predictions_path
        transformer_predictions_path = transformer_mimic_death_in_hospital_predictions_path
    else:
        raise ValueError('Unknown outcome')

    lstm_gt, lstm_predictions = pickle.load(open(lstm_predictions_path, 'rb'))
    transformer_gt, transformer_predictions = pickle.load(open(transformer_predictions_path, 'rb'))
    print(lstm_gt.shape, lstm_predictions.shape, transformer_predictions.shape)

    auc_lstm, auc_transformer, p_value = get_auc_and_p_value(lstm_gt, lstm_predictions.squeeze(), transformer_predictions)
    return auc_lstm, auc_transformer, p_value

df = pd.DataFrame(columns=['Outcome', 'LSTM AUC', 'Transformer AUC', 'p-value'])
for outcome in ['Death in hospital', '3M Death', '3M mRS 0-2', 'MIMIC Death in hospital','MIMIC 3M Death']:
    auc_lstm, auc_transformer, p_value = get_auc_and_p_value_for_outcome_lstm(outcome)
    df = df.append({'Outcome': outcome, 'LSTM AUC': auc_lstm, 'Transformer AUC': auc_transformer, 'p-value': p_value[0][0]}, ignore_index=True)

In [ ]:
df

In [ ]:
# df.to_csv('/Users/jk1/Downloads/model_comparison_lstm_transformer.csv')

# Compare THRIVE-C with transformer

In [ ]:
def get_auc_and_p_value_for_outcome_thrive(outcome):
    if outcome == '3M Death':
        thrive_predictions_path = thrivec_death_predictions_path
        transformer_predictions_path = transformer_death_predictions_path
    elif outcome == '3M mRS 0-2':
        thrive_predictions_path = thrivec_mrs_predictions_path
        transformer_predictions_path = transformer_mrs_predictions_path
    else:
        raise ValueError('Unknown outcome')

    thrive_gt, thrive_predictions = pickle.load(open(thrive_predictions_path, 'rb'))
    transformer_gt, transformer_predictions = pickle.load(open(transformer_predictions_path, 'rb'))

    auc_thrive, auc_transformer, p_value = get_auc_and_p_value(thrive_gt, thrive_predictions.squeeze(), transformer_predictions)
    return auc_thrive, auc_transformer, p_value

df = pd.DataFrame(columns=['Outcome', 'THRIVE-C AUC', 'Transformer AUC', 'p-value'])
for outcome in ['3M Death', '3M mRS 0-2']:
    auc_thrive, auc_transformer, p_value = get_auc_and_p_value_for_outcome_thrive(outcome)
    df = df.append({'Outcome': outcome, 'THRIVE-C AUC': auc_thrive, 'Transformer AUC': auc_transformer, 'p-value': p_value[0][0]}, ignore_index=True)